In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, BorderlineSMOTE
from collections import Counter

%matplotlib inline
sns.set(style='whitegrid')

In [ ]:
def plot_class_distribution(y, labels, title):
    counts = pd.Series(y).value_counts().sort_index()
    valid_indices = counts.index[counts.index < len(labels)]
    counts = counts.loc[valid_indices]
    names = labels[counts.index]

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x=names, y=counts.values, hue=names, palette='viridis', legend=False)
    ax.set_title(title)
    ax.set_xlabel('Sınıf')
    ax.set_ylabel('Sayı')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
def load_data():
    tracks_path = 'fma_metadata/tracks.csv'
    features_path = 'fma_metadata/features.csv'

    if not os.path.exists(tracks_path) or not os.path.exists(features_path):
        raise FileNotFoundError(f"Gerekli veri dosyaları bulunamadı. '{tracks_path}' ve '{features_path}' dosyalarının mevcut olduğundan emin olun.")

    tracks = pd.read_csv(tracks_path, index_col=0, header=[0,1])
    
    features = pd.read_csv(features_path, index_col=0, header=[0,1])  # Çok seviyeli başlıkla oku
    features = features.loc[:, features.columns.get_level_values(0) != 'statistics']  # 'statistics' sütunlarını kaldır
    features = features.astype(np.float32)  # Sayısal olmayan sütunları kaldırdıktan sonra float'a dönüştür

    features.index = features.index.astype(str)
    tracks.index = tracks.index.astype(str)

    genre_series = tracks[('track', 'genre_top')].dropna()
    common_index = features.index.intersection(genre_series.index)

    X = features.loc[common_index]
    y_labels = genre_series.loc[common_index]

    X = X.fillna(0).replace([np.inf, -np.inf], 0).astype(np.float32)

    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y_labels)

    print('Veriler yüklendi ve önişlendi.')
    return X, y, label_encoder

In [ ]:
# Veriyi yükle ve önişle
X, y, le = load_data()

# Başlangıç dağılımını göster
plot_class_distribution(y, le.classes_, 'Başlangıç Sınıf Dağılımı')

In [ ]:
# Veriyi böl ve eğitim dağılımını göster
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

plot_class_distribution(y_train, le.classes_, 'Eğitim Seti Dağılımı')
print(f'Eğitim/test bölünmesi tamamlandı: X_train {X_train.shape}, X_test {X_test.shape}')

# Detaylı dağılımı yazdır
unique, counts = np.unique(y_train, return_counts=True)
print("\nEğitim Seti Dağılımı (ham sayılar):")
for i, (u, c) in enumerate(zip(unique, counts)):
    print(f"Sınıf {u} ({le.classes_[i]}): {c} örnek")

In [ ]:
# Adım 1: En az temsil edilen sınıflar için RandomOverSampler
print('\nAdım 1: Aşırı az temsil edilen sınıflar için RandomOverSampler uygulanıyor...')
min_samples_threshold = 100  # BorderlineSMOTE için gereken minimum örnek sayısı
ros = RandomOverSampler(sampling_strategy={3: min_samples_threshold}, random_state=42)
X_partial, y_partial = ros.fit_resample(X_train, y_train)

# Ara sonuçları göster
unique_partial, counts_partial = np.unique(y_partial, return_counts=True)
print("\nRandomOverSampler sonrası dağılım (ham sayılar):")
for i, (u, c) in enumerate(zip(unique_partial, counts_partial)):
    print(f"Sınıf {u} ({le.classes_[i]}): {c} örnek")

plot_class_distribution(y_partial, le.classes_, 'RandomOverSampler Sonrası')

In [ ]:
# Adım 2: Kalan sınıflar için BorderlineSMOTE
print('\nAdım 2: Kalan sınıflar için BorderlineSMOTE uygulanıyor...')
borderline_smote = BorderlineSMOTE(random_state=42)

try:
    X_res, y_res = borderline_smote.fit_resample(X_partial, y_partial)
    print(f'Kombine örnekleme tamamlandı: X_res {X_res.shape}, y_res {y_res.shape}')
    
    # Son dağılımı yazdır ve göster
    unique_res, counts_res = np.unique(y_res, return_counts=True)
    print("\nSon Dağılım (ham sayılar):")
    for i, (u, c) in enumerate(zip(unique_res, counts_res)):
        print(f"Sınıf {u} ({le.classes_[i]}): {c} örnek")
    
    plot_class_distribution(y_res, le.classes_, 'Son Dengelenmiş Dağılım')

except Exception as e:
    print(f'BorderlineSMOTE örnekleme başarısız oldu: {e} - kısmi örneklenmiş veri kullanılıyor')
    X_res, y_res = X_partial, y_partial
    plot_class_distribution(y_res, le.classes_, 'Kısmi Örnekleme (BorderlineSMOTE başarısız)')

print("\nİşlem hattı tamamlandı. Yeniden örneklenmiş eğitim verisi (X_res, y_res) ve test verisi (X_test, y_test) hazır.")